In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import math

# Define the denoising autoencoder model
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=4),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Load the pre-trained model
model = DenoisingAutoencoder()
model.load_state_dict(torch.load('denoising_model_2.pth'))
model.eval()

# Define data transformation for inference
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
])

# Load and preprocess the original and noisy images
original_image_path = 'C:\\Satellite-Image-Denoising-master\\Data\\Train\\Clean\\class_1\\1_0.jpeg'
noisy_image_path = 'C:\\Satellite-Image-Denoising-master\\Data\\Train\\Noisy\\class_1\\1_0.jpeg'


original_image = Image.open(original_image_path).convert('L')
noisy_image = Image.open(noisy_image_path).convert('L')

original_tensor = transform(original_image).unsqueeze(0)  # Add batch dimension
noisy_tensor = transform(noisy_image).unsqueeze(0)  # Add batch dimension

# Calculate PSNR before denoising
def calculate_psnr(image1, image2):
    mse = ((image1 - image2) ** 2).mean()
    max_pixel_value = 255
    psnr = 20 * math.log10(max_pixel_value) - 10 * math.log10(mse)
    return psnr

psnr_before_denoising = calculate_psnr(original_tensor.squeeze(0).numpy() * 255, noisy_tensor.squeeze(0).numpy() * 255)

# Perform denoising
with torch.no_grad():
    denoised_tensor = model(noisy_tensor)

# Calculate PSNR after denoising
psnr_after_denoising = calculate_psnr(original_tensor.squeeze(0).numpy() * 255, denoised_tensor.squeeze(0).numpy() * 255)

print(f"PSNR before denoising: {psnr_before_denoising:.2f} dB")
print(f"PSNR after denoising: {psnr_after_denoising:.2f} dB")

# Convert the denoised tensor to a PIL image
denoised_image = transforms.ToPILImage()(denoised_tensor.squeeze(0))

# Save the original and noisy images for comparison
original_image.save('original_image1.jpeg')
noisy_image.save('noisy_image1.jpeg')

# Save the denoised image
denoised_image.save('denoised_image1.jpeg')




C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2556235396.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('denoising_model_2.pth'))


PSNR before denoising: 12.95 dB
PSNR after denoising: 19.80 dB
